In [20]:
import pandas as pd

# Path for CMU Data
PATH = "/work/ada-2022-project-northamericans/Data/MovieSummaries/"
pd.set_option('display.max_rows', 0)
pd.set_option('display.max_columns', 0)


pd.set_option('display.width', 0)
pd.set_option('max_colwidth', 0)

In [21]:
# Column names as columns are not labeled in the TSV files
columns = {
    "plot_summaries.txt" : ["id", "summary"],
    "character.metadata.tsv": ["Wikipedia Movie ID", "Freebase Movie ID", "Movie Release Date", 
                               "Character Name", "Actor DOB", "Actor gender", "Actor height (in meters)", 
                               "Actor ethnicity (Freebase ID)", "Actor Name", "Actor age at movie release", 
                               "Freebase character/actor map ID", "Freebase character ID", "Freebase actor ID"],
    "movie.metadata.tsv": ["Wikipedia movie ID", "Freebase movie ID", "Movie name", "Movie release date", 
                           "Movie box office revenue", "Movie runtime", 
                           "Movie languages (Freebase ID:name tuples)", 
                           "Movie countries (Freebase ID:name tuples)", 
                           "Movie genres (Freebase ID:name tuples)"]
}

In [22]:
dataframes = {}

In [23]:
# Read the dataframes into a dictionary with mapping filename -> dataframe
# Labelled the columns defined earlier
for key, value in columns.items():
    df = pd.read_csv(f"{PATH}{key}", sep="\t", names=value)
    dataframes[key] = df

In [24]:
for key, value in dataframes.items():
    print(key, value.shape)
# Shape of data we are working with

plot_summaries.txt (42303, 2)
character.metadata.tsv (450669, 13)
movie.metadata.tsv (81741, 9)


In [25]:
dataframes["character.metadata.tsv"].sort_values("Movie Release Date").dropna()

dataframes["character.metadata.tsv"].head()

,Wikipedia Movie ID,Freebase Movie ID,Movie Release Date,Character Name,Actor DOB,Actor gender,Actor height (in meters),Actor ethnicity (Freebase ID),Actor Name,Actor age at movie release,Freebase character/actor map ID,Freebase character ID,Freebase actor ID
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg


In [26]:
dataframes["movie.metadata.tsv"].head()

,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages (Freebase ID:name tuples),Movie countries (Freebase ID:name tuples),Movie genres (Freebase ID:name tuples)
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science Fiction"", ""/m/03npn"": ""Horror"", ""/m/03k9fj"": ""Adventure"", ""/m/0fdjb"": ""Supernatural"", ""/m/02kdv5l"": ""Action"", ""/m/09zvmj"": ""Space western""}"
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey Mystery,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0"": ""Drama"", ""/m/0hj3n01"": ""Crime Drama""}"
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""Drama""}"
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic thriller"", ""/m/09blyk"": ""Psychological thriller""}"
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


In [27]:
dataframes["plot_summaries.txt"].head()


id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [28]:
# Basic info of the datasets regarding actors and movies.
print(f"Unique movies: {len(dataframes['movie.metadata.tsv']['Movie name'].unique())}")
print(f"Unique actors: {len(dataframes['character.metadata.tsv']['Actor Name'].unique())}")
print(f"Unique characters: {len(dataframes['character.metadata.tsv']['Character Name'].unique())}")


Unique movies: 75478
Unique actors: 134079
Unique characters: 126630


In [11]:
# Coreference Resolution Pipeline
import coreferee, spacy

# Load NLP model into spacy
nlp = spacy.load('en_core_web_trf')
nlp.add_pipe('coreferee')

# Method we apply on every summary to get the coreference resolution and clean the output
def method(s):
    doc = nlp(s)
    stuff = doc._.coref_chains
    list_stuff = stuff.pretty_representation.split(';')
    list_stuff = [i for i in list_stuff if "[" not in i]
    list_stuff = [i[i.find(":")+1::].strip().split(",") for i in list_stuff]
    list_stuff = [set([i[0:i.find("(")].strip() for i in j]) for j in list_stuff]
    return list_stuff

# Looks similar to this, where we see the occurences of each reference that we can aggregate later
# for multiple occurences of the same characters. The number refers to the ith word in the summary.
''' 
['0: survivor(58), her(67), Her(84), her(91), she(107)',
 ' 1: District(81), District(114)',
 ' 2: Katniss(87), Katniss(104), Katniss(118), Katniss(173)',
 ' 3: [Katniss(118)',
 ' Peeta(120)], their(129), them(141)',
 ' 4: Peeta(120), Peeta(167), his(170), Peeta(212), he(215), Peeta(263)',
 ' 5: Abernathy(137), He(139)',
 ' 6: Flickerman(165), he(175), he(210)', ...
 '''

In [33]:
# This code reads a summary from the dataframe, and then calculates the coreferences, and then we write
# them to disk. This is because we use remote computer for our work, where we have access to limited
# amount of ram.
for i, (index, row) in tqdm(enumerate(dataframes["plot_summaries.txt"].iterrows())):
    # if i < 29000: 
    #     continue # skip first 5 rows
    data = method(row["summary"])
    # Pickle the datastructure to disk so that we can retrieve the object later
    with open(f"/work/ada-2022-project-northamericans/Data/coreferences_pickled/{row['id']}.pkl", "wb") as f:
        pickle.dump(data, f)

In [34]:
import glob
import pickle

# Read the files so that we can aggregate the occurences in each summary post coreference resolution.

files = glob.glob("/work/ada-2022-project-northamericans/Data/coreferences_pickled/*")
arr = []
len(files)
# for filename in files:
#     with open(filename, "rb") as f:
#         data = pickle.load(f)
#         arr.append(data)
arr = []
for filename in files:
    with open(filename, "rb") as f:
        data = pickle.load(f)
        arr.append(data)

    

In [35]:
# specify relevant column names and data path
DATA_PATH = "/work/ada-2022-project-northamericans/Data/"
CMU_COLUMN_NAMES = ["Wikipedia Movie ID", "Freebase Movie ID", "Movie Release Date", 
                    "Character Name", "Actor DOB", "Actor gender", "Actor height (in meters)", 
                    "Actor ethnicity (Freebase ID)", "Actor Name", "Actor age at movie release", 
                    "Freebase character/actor map ID", "Freebase character ID", "Freebase actor ID"]
IMDB_COLUMN_NAMES = ["nconst", "primaryName", "birthYear", "deathYear", "primaryProfession", "knownForTitles"]
IMDB_COLUMN_TYPES = {"nconst": str,
                     "primaryName": str,
                     "birthYear": object,
                     "deathYear": object,
                     "primaryProfession": str,
                     "knownForTitles": str}


cmu_actors_df = pd.read_csv(DATA_PATH + 'MovieSummaries/character.metadata.tsv', 
                            sep="\t", 
                            names=CMU_COLUMN_NAMES, 
                            error_bad_lines=False)

# create dataframe from IMDB character data, done in chunks to avoid overwhelming deepnote machine
imdb_actors_df = pd.DataFrame(columns=IMDB_COLUMN_NAMES)
idx = 0
for chunk in pd.read_csv(DATA_PATH + 'IMDB/name.basics.tsv.gz', 
                         compression='gzip', sep='\t', 
                         dtype=IMDB_COLUMN_TYPES, 
                         chunksize=1000000, 
                         error_bad_lines=False):
    print(f'On chunk {idx}')
    idx += 1

    # contains \N when unknown or not relevent. we can set birthYear to 0 and remove
    chunk['birthYear'] = chunk['birthYear'].replace([r'\N'], 0).astype(int)
    # but deathYear could be \N because they haven't died yet. so set to 9999 so it isn't filtered out
    chunk['deathYear'] = chunk['deathYear'].replace([r'\N'], 9999).astype(int)

    chunk = chunk[chunk['birthYear'] > 1900]
    chunk = chunk[chunk['deathYear'] > 2012]
    chunk = chunk[chunk['primaryProfession'].str.contains('actor') | 
                  chunk['primaryProfession'].str.contains('actress')]

    imdb_actors_df = imdb_actors_df.append(chunk)

imdb_actors_df.head()

On chunk 0
On chunk 1
On chunk 2
On chunk 3
On chunk 4
On chunk 5
On chunk 6
On chunk 7
On chunk 8
On chunk 9
On chunk 10
On chunk 11
On chunk 12


,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0071877,tt0037382,tt0038355,tt0117057"
2,nm0000003,Brigitte Bardot,1934,9999,"actress,soundtrack,music_department","tt0049189,tt0056404,tt0057345,tt0054452"
12,nm0000013,Doris Day,1922,2019,"soundtrack,actress,producer","tt0045591,tt0053172,tt0048317,tt0049470"
13,nm0000014,Olivia de Havilland,1916,2020,"actress,soundtrack","tt0041452,tt0040806,tt0031381,tt0029843"
17,nm0000018,Kirk Douglas,1916,2020,"actor,producer,soundtrack","tt0049456,tt0043338,tt0080736,tt0054331"


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f72b9652-2157-4879-b741-0567715b1f22' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>